In [2]:
# MAIN CODE: Agent Tracker Automation
# This code will look for an excel workbook named Master.xlsx which contains the source data to be analyzed
# Then will output in another workbook named AT.xlsx using new sheets for every generation


import openpyxl
import os
from datetime import datetime
import subprocess

# Function to process data and write counts to the 'AT' workbook
def process_and_write_counts(master_file, at_file):
    try:
        print("Loading the Master and AT workbooks...")

        # Load the 'Master.xlsx' workbook and 'AT.xlsx' workbook
        master_wb = openpyxl.load_workbook(master_file)
        at_wb = openpyxl.load_workbook(at_file)

        # Load the 'Form1' sheet from the 'Master.xlsx' file
        master_sheet = master_wb['Form1']  # Using 'Form1' as the sheet name

        # Create a new sheet in the 'AT' workbook with a timestamp in the name (to avoid overwriting)
        timestamp = datetime.now().strftime('%Y_%m_%d')
        new_sheet_name = f"AT_{timestamp}"
        at_sheet = at_wb.create_sheet(new_sheet_name)

        # Add headers to the new sheet
        headers = [
            "Agent", "TL", "Analyst", "Last Update", "Greeting", "Verif", "Ack", "Ownership & Willingness to Assist",
            "Pos Phr/Tone", "Empathy Stmts", "Actv Listng", "Mandatory Questions Heard", "Call Control", "SMS Opt-in",
            "Used Tools", "De-escalation skills", "Esc to Ldr for aggr members", "Rapport-building efforts", "Dead air",
            "Proper Hold Process", "Transfr Expcts/ Propr Transfr Proc", "Additional Assistance", "Call Recap",
            "Branded Closing", "No Work Avoidance?", "Five9: Dispo", "Five9: Notes Acc/Detail", "Documented STC?",
            "Correct Verbiage"
        ]
        for col_idx, header in enumerate(headers, start=1):
            at_sheet.cell(row=1, column=col_idx, value=header)

        print("Reading agent names and processing data...")

        # Initialize agent names list and counts dictionary
        agent_data = {}

        # Get the current month and year
        current_month = datetime.now().month
        current_year = datetime.now().year

        # Set the valid emails
        valid_emails = {
            "daniel.suarez@365roi.com",
            "jansen.machado@365roi.com",
            "justin.dennis@365roi.com"
        }

        # Reading agent names (column I) and filtering by date range (column C)
        for row in master_sheet.iter_rows(min_row=2, max_row=master_sheet.max_row, min_col=9, max_col=9):  # Column I is column 9
            agent_name = row[0].value
            date_value = master_sheet.cell(row=row[0].row, column=3).value  # Column C

            # Apply the date filter
            if date_value and isinstance(date_value, datetime):
                if date_value.month != current_month or date_value.year != current_year:
                    continue  # Skip if it's not in the current month
            else:
                continue  # Skip if the date is invalid or missing

            if agent_name:
                agent_name = str(agent_name)  # Ensure agent names are treated as strings
                agent_data[str(agent_name)] = [None, None, None, None] + [None] * 25  # Prepare a list for extra columns and counts (25 columns L to AJ)

        print("Processing rows for 'No' counts...")

        # Now process columns L to AJ for 'No' counts and apply the filters
        for row in master_sheet.iter_rows(min_row=2, max_row=master_sheet.max_row, min_col=12, max_col=36):  # Columns L to AJ are columns 12 to 36
            # Get the values from columns C, D, E, H, J
            date_value = master_sheet.cell(row=row[0].row, column=3).value  # Column C
            email_value = master_sheet.cell(row=row[0].row, column=4).value  # Column D
            additional_data = master_sheet.cell(row=row[0].row, column=5).value  # Column E
            monitor_value = master_sheet.cell(row=row[0].row, column=8).value  # Column H
            secondary_name = master_sheet.cell(row=row[0].row, column=10).value  # Column J

            # Apply the date filter again
            if date_value and isinstance(date_value, datetime):
                if date_value.month != current_month or date_value.year != current_year:
                    continue  # Skip if it's not in the current month
            else:
                continue  # Skip if the date is invalid or missing

            # 2. Check if "Monitor" is in column H (case-sensitive check)
            if monitor_value != "Monitor":
                continue  # Skip if it's not "Monitor"

            # 3. Check if the email is one of the valid emails (strip spaces before comparison)
            if email_value and email_value.strip() not in valid_emails:
                continue  # Skip if the email is not in the valid emails

            # Get agent name from column I
            agent_name = master_sheet.cell(row=row[0].row, column=9).value  # Column I
            first_name = additional_data.split()[0] if additional_data else ""  # Extract first name only from column E

            if agent_name and agent_name in agent_data:
                agent_name = str(agent_name)  # Ensure agent names are treated as strings
                # Set agent primary name, secondary name, additional data, and date
                agent_data[agent_name][0] = agent_name  # Column A: Primary agent name
                agent_data[agent_name][1] = secondary_name  # Column B: Secondary name (from column J)
                agent_data[agent_name][2] = first_name  # Column C: First name only from column E
                agent_data[agent_name][3] = datetime.now().strftime('%d-%b')  # Column D: Date in Day-Month format
                # Count "No" entries in columns L to AJ
                for idx, cell in enumerate(row):
                    if cell.value == "No" and idx < 25:
                        agent_data[agent_name][idx + 4] = (agent_data[agent_name][idx + 4] or 0) + 1  # Accumulate counts

        print("Writing counts to the new sheet...")

        # Write agent names and counts into the new sheet
        agent_names = sorted([name for name in agent_data if any(agent_data[name])])  # Filter out empty rows
        row_idx = 2  # Starting from row 2 in the new sheet (row 1 is for headers)
        for agent_name in agent_names:
            for col_idx, value in enumerate(agent_data[agent_name]):
                if value is not None:
                    at_sheet.cell(row=row_idx, column=col_idx + 1, value=value)
            row_idx += 1  # Move to the next row for the next agent

        # Save the 'AT' workbook with the new sheet
        at_wb.save(at_file)

        # Print success message after saving the workbook
        print(f"Output successfully written to {at_file}, sheet '{new_sheet_name}'")

        # Prompt to open the destination file and take the user to the new sheet
        open_choice = input(f"Do you want to open the destination file '{at_file}'? (Y/N): ").strip().lower()
        if open_choice == 'y':
            subprocess.run(['start', at_file], shell=True)  # Open the file with the default associated program
            print(f"Opening {at_file}...")

    except PermissionError:
        print("Permission denied: Make sure the file is not open or being used by another program.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Get the current working directory where the script and workbooks are located
folder_path = os.getcwd()

# Define file paths based on the current folder
master_file = os.path.join(folder_path, 'Master.xlsx')
at_file = os.path.join(folder_path, 'AT.xlsx')

# Process and write the data
process_and_write_counts(master_file, at_file)

Loading the Master and AT workbooks...
Reading agent names and processing data...
Processing rows for 'No' counts...
Writing counts to the new sheet...
Output successfully written to C:\Users\Remote\Documents\Automation\AT.xlsx, sheet 'AT_2025_02_13'


Do you want to open the destination file 'C:\Users\Remote\Documents\Automation\AT.xlsx'? (Y/N):  y


Opening C:\Users\Remote\Documents\Automation\AT.xlsx...


In [2]:
# SECONDARY CODE
# This code will look for Verification misses and output a list

import openpyxl
import os
from datetime import datetime

# Function to check column M for "No" and copy specified columns to a new sheet
def check_column_m_and_copy(master_file, at_file):
    try:
        print("Loading the Master and AT workbooks...")

        # Load the 'Master.xlsx' workbook and 'AT.xlsx' workbook
        master_wb = openpyxl.load_workbook(master_file)
        at_wb = openpyxl.load_workbook(at_file)

        # Load the 'Form1' sheet from the 'Master.xlsx' file
        master_sheet = master_wb['Form1']  # Using 'Form1' as the sheet name

        # Create a new sheet in the 'AT' workbook named "Verif Errors MTD"
        new_sheet_name = "Verif Errors MTD"
        at_sheet = at_wb.create_sheet(new_sheet_name)

        # Add headers to the new sheet
        headers = [
            "ID", "Submitter", "Date Of Call", "Type of Review", "Name (Agent)", "Name of TL", "CID", "Wins", 
            "Opportunities", "Additional Feedback", "Score", "Issue", "Coaching Status", "Side Note"
        ]
        for col_idx, header in enumerate(headers, start=1):
            at_sheet.cell(row=1, column=col_idx, value=header)

        print("Processing rows for 'No' in column M...")

        # Get the current month and year
        current_month = datetime.now().month
        current_year = datetime.now().year

        # List to store rows to be copied
        rows_to_copy = []

        # Iterate over each row in the master sheet
        for row in master_sheet.iter_rows(min_row=2, max_row=master_sheet.max_row):
            date_value = row[2].value  # Column C is index 2
            if date_value and isinstance(date_value, datetime):
                if date_value.month == current_month and date_value.year == current_year:
                    if row[12].value == 'No':  # Column M is index 12
                        # Copy specified columns to the list
                        columns_to_copy = [0, 4, 6, 7, 8, 9, 10, 36, 37, 39, 38]  # Columns A, E, G, H, I, J, K, AK, AL, AN, AM
                        row_data = []
                        for col_idx, master_col_idx in enumerate(columns_to_copy):
                            cell_value = row[master_col_idx].value
                            if col_idx == 2:  # Column C (Date Of Call) should be in short date format
                                if isinstance(cell_value, datetime):
                                    cell_value = cell_value.strftime('%m/%d/%Y')
                            row_data.append(cell_value)
                        row_data.append("Verification")  # Add "Verification" to the Issue column
                        rows_to_copy.append(row_data)

        # Sort rows by Date Of Call (index 2)
        rows_to_copy.sort(key=lambda x: datetime.strptime(x[2], '%m/%d/%Y'))

        # Write sorted rows to the new sheet
        for row_idx, row_data in enumerate(rows_to_copy, start=2):
            for col_idx, cell_value in enumerate(row_data, start=1):
                at_sheet.cell(row=row_idx, column=col_idx, value=cell_value)

        # Save the 'AT' workbook with the new sheet
        at_wb.save(at_file)

        print(f"Output successfully written to {at_file}, sheet '{new_sheet_name}'")

    except PermissionError:
        print("Permission denied: Make sure the file is not open or being used by another program.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Get the current working directory where the script and workbooks are located
folder_path = os.getcwd()

# Define file paths based on the current folder
master_file = os.path.join(folder_path, 'Master.xlsx')
at_file = os.path.join(folder_path, 'AT.xlsx')

# Process and write the data
check_column_m_and_copy(master_file, at_file)

Loading the Master and AT workbooks...
Processing rows for 'No' in column M...
Output successfully written to C:\Users\Remote\Documents\Automation\AT.xlsx, sheet 'Verif Errors MTD'
